In [20]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import os.path
from os import listdir
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
import seaborn as sns
import matplotlib.pyplot as plt
import py_entitymatching as em
import py_stringmatching as sm
import scipy
from scipy.io import arff
from sklearn import preprocessing
#from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

In [21]:
#read table to dataframe and create train/test split with equal distribution of labels within table
all_tables = pd.read_csv('../../src/data/data/CSV_files/relevant_tables_more_than_20.csv')
all_tables = all_tables.drop(columns={'Unnamed: 0'})
#pd.set_option('display.max_columns', None)
all_tables

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,...,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries
0,Book_12min.com_September2020.json.gz,Book,366.0,9.0,5.0,166,0.286885,0.090710,830.0,1,...,0,0,0,0,0,0,0,0,0,0
1,Book_19shuwu.com_September2020.json.gz,Book,250.0,8.0,5.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,Book_1carpetcleaning.co.uk_September2020.json.gz,Book,1004.0,12.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,Book_2014brazil.co.uk_September2020.json.gz,Book,387.0,11.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0,0,0
4,Book_24symbols.com_September2020.json.gz,Book,5171.0,8.0,5.0,17,0.001740,0.000658,85.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54185,TVEpisode_yara-online.org_September2020.json.gz,TVEpisode,40.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,1,1
54186,TVEpisode_yesmoviess.com_September2020.json.gz,TVEpisode,37.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,1,1
54187,TVEpisode_zazangels.com_September2020.json.gz,TVEpisode,38.0,8.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,1,1
54188,TVEpisode_zebrahead.org_September2020.json.gz,TVEpisode,80.0,7.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,1,1


In [45]:
all_tables.loc[all_tables['num_rows'] <= 40, 'Small'] = 1 
all_tables.loc[(all_tables['num_rows'] > 40) & (all_tables['num_rows'] <= 160), 'Medium'] = 1
all_tables.loc[all_tables['num_rows'] > 160, 'Large'] = 1 
all_tables.fillna(0)

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,...,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries,Small,Medium,Large
0,Book_12min.com_September2020.json.gz,Book,366.0,9.0,5.0,166,0.286885,0.090710,830.0,1,...,0,0,0,0,0,0,0,0.0,0.0,1.0
1,Book_19shuwu.com_September2020.json.gz,Book,250.0,8.0,5.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
2,Book_1carpetcleaning.co.uk_September2020.json.gz,Book,1004.0,12.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0.0,0.0,1.0
3,Book_2014brazil.co.uk_September2020.json.gz,Book,387.0,11.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0.0,0.0,1.0
4,Book_24symbols.com_September2020.json.gz,Book,5171.0,8.0,5.0,17,0.001740,0.000658,85.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54185,TVEpisode_yara-online.org_September2020.json.gz,TVEpisode,40.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,1.0,0.0,0.0
54186,TVEpisode_yesmoviess.com_September2020.json.gz,TVEpisode,37.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,1.0,0.0,0.0
54187,TVEpisode_zazangels.com_September2020.json.gz,TVEpisode,38.0,8.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,1.0,0.0,0.0
54188,TVEpisode_zebrahead.org_September2020.json.gz,TVEpisode,80.0,7.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,0.0,1.0,0.0


In [73]:
 liste  = list(all_tables.iloc[:,1:2])
 liste2 = list(all_tables.iloc[:,9:])
 liste = liste + liste2
 liste

['Class',
 'Book_aggregaterating',
 'Book_author',
 'Book_bookedition',
 'Book_bookformat',
 'Book_datepublished',
 'Book_genre',
 'Book_inlanguage',
 'Book_isbn',
 'Book_name',
 'Book_numberofpages',
 'Book_offers',
 'Book_publisher',
 'CreativeWork_aggregaterating',
 'CreativeWork_alternativeheadline',
 'CreativeWork_articlebody',
 'CreativeWork_author',
 'CreativeWork_commentcount',
 'CreativeWork_copyrightholder',
 'CreativeWork_copyrightyear',
 'CreativeWork_creator',
 'CreativeWork_datecreated',
 'CreativeWork_datemodified',
 'CreativeWork_datepublished',
 'CreativeWork_genre',
 'CreativeWork_headline',
 'CreativeWork_inlanguage',
 'CreativeWork_interactioncount',
 'CreativeWork_interactionstatistic',
 'CreativeWork_interactiontype',
 'CreativeWork_keywords',
 'CreativeWork_mainentityofpage',
 'CreativeWork_name',
 'CreativeWork_offers',
 'CreativeWork_publisher',
 'CreativeWork_version',
 'Event_doortime',
 'Event_duration',
 'Event_enddate',
 'Event_eventattendancemode',
 'Even

In [78]:
all_tables = all_tables.fillna(0)

In [74]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'
tables = pd.read_csv(path + '/all_files_cleaned.csv')
tables = tables.drop(columns={'Unnamed: 0'})
columns_sorted = tables.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
columns_sorted.filename = columns_sorted.filename.str.split('_', expand = True)
columns_sorted.index = columns_sorted['filename']
columns_sorted

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepublished, numberofpages, bookformat, inlanguage, ge..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified, name, publisher, mainentityofpage, interactionst..."
Event,Event,"[name, enddate, location, performer, organizer, eventstatus, eventattendancemode, duration, door..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, aggregaterating, starrating, email]"
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, aggregaterating, email, sameas, faxnumber, openingho..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublished, genre, datecreated]"
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenname, familyname, telephone, email, deathdate, addi..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspecification]"


In [79]:
#Split all Tables in large train and test set
#def traintestsplit():
#for index in columns_sorted.filename:
#    index_df = pd.DataFrame()
#    liste = []
#    index_df = small[small['Class']==index]
    #display(index_df)
#    liste  = list([col for col in small if col.startswith(index+'_')])
#liste = list([col for col in small if col.startswith(index+'_')])
X = np.array(all_tables['filename'])
y = np.array(all_tables[liste])
msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.18, random_state=42)
for train_index, test_index in msss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#print(test_index)
train_set = set(X_train)
print(train_set)
test_set = set(X_test)
print(test_set)
for index in columns_sorted.filename:
    inputdir = '../../src/data/schemafiltereddata/'+index+'/'
    outputdir = '../../src/data/schemafiltereddata/TrainTestTables/Large/'
    files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]  
    for file in files:
        if file in train_set:
            #for file in train_set:
            #print(file)
            df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
            df.to_json(os.path.join(outputdir+'Train/'+file), compression='gzip', orient='records', lines=True)
        print(file)
        if file in test_set:
        #for file in test_set:
            #print(file)
            df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
            df.to_json(os.path.join(outputdir+'Test/'+file), compression='gzip', orient='records', lines=True)
        print(file)
print(len([file for file in os.listdir(outputdir+'Test/') if file.endswith('.json.gz')]))

TRAIN: [    0     1     3 ... 54187 54188 54189] TEST: [    2    15    18 ... 54177 54184 54186]
TRAIN: [    0     2     3 ... 54184 54187 54189] TEST: [    1     6    13 ... 54185 54186 54188]
TRAIN: [    0     2     3 ... 54186 54187 54189] TEST: [    1     7     9 ... 54176 54182 54188]
